# Continual Learning for pedestrian detection (CrowdHuman to CityScape)

In [4]:
import torch
cuda_enable = True
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU")
else:
  device = torch.device("cpu")
  print("CPU")

GPU


In [5]:
import os
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn
from pycocotools.coco import COCO
from torchvision import datasets, transforms, models

In [6]:
class ImageDataset(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms=None):
    self.root = root
    self.transforms = transforms
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["file_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))
    # get annotation ID
    ann_ids = self.coco.getAnnIds(imgIds = img_id)
    # read annotation
    anns = self.coco.loadAnns(ann_ids)
    # num of people in the picture
    num_objs = len(anns)
    # build information about bounding box & area
    boxes = []
    areas = []
    for i in range(num_objs):
      x_min = anns[i]['bbox'][0]
      y_min = anns[i]['bbox'][1]
      x_max = x_min + anns[i]['bbox'][2]
      y_max = y_min + anns[i]['bbox'][3]
      boxes.append([x_min, y_min, x_max, y_max])
  
    # transfer information to Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.ones((num_objs,), dtype = torch.int64)
    img_id = torch.tensor([img_id])
    areas = torch.as_tensor(areas, dtype=torch.float32)
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    # Annotation in dict form
    Annotations = {
        "boxes" : boxes,
        "labels" : labels,
        "image_id" : img_id,
        "iscrowd" : iscrowd
    }

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img, Annotations
  
  def __len__(self):
    return len(self.ids)

In [7]:
def get_transforms(train):
  trans = []
  if train:
    trans.append(transforms.RandomHorizontalFlip(0.5))
    # trans.append(transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)))
    # trans.append(transforms.ColorJitter(brightness=1, contrast=1, saturation=1))
  trans.append(transforms.ToTensor())
  return transforms.Compose(trans)

train_data_path = r'data\CrowdHuman\images'
coco_path = r"C:\Users\mahdi\Desktop\Pedestrian-Detection-master\data\CrowdHuman\train.json"
train_dataset = ImageDataset(root=train_data_path, annotation=coco_path, transforms=get_transforms(train=True))
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))
    
train_loader = torch.utils.data.DataLoader(train_dataset, 1, shuffle=True, collate_fn=collate_fn)

loading annotations into memory...
Done (t=2.39s)
creating index...
index created!


In [8]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# change question to binary classification (human, not human)
num_classes = 2

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [9]:
#Load the weights of the model
model.load_state_dict(torch.load('ckpt/FRCNN_crowdhuman.pth'))
model = model.to(device)

In [10]:
#test data loader
class ImageDataset_test(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms):
    self.root = root
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))
    self.transforms = transforms

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["im_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img
  
  def __len__(self):
    return len(self.ids)

test_data_path = r'data\citypersons\images\val'
coco_path_test = r"data\citypersons\annotations\val_gt.json"
test_dataset = ImageDataset_test(root=test_data_path, annotation=coco_path_test, transforms=get_transforms(train=False))

    
test_loader_city = torch.utils.data.DataLoader(test_dataset, 1, shuffle=False)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [11]:
#test data loader
class ImageDataset_test(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms):
    self.root = root
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))
    self.transforms = transforms

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["file_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img
  
  def __len__(self):
    return len(self.ids)

test_data_path = r'data\CrowdHuman\images\val'
coco_path_test = r"data\CrowdHuman\val.json"
test_dataset = ImageDataset_test(root=test_data_path, annotation=coco_path_test, transforms=get_transforms(train=False))

    
test_loader_crowd = torch.utils.data.DataLoader(test_dataset, 1, shuffle=False)

loading annotations into memory...
Done (t=0.68s)
creating index...
index created!


In [12]:
import sys
import json
from IPython.display import clear_output

def log_output(model,test_loader,epoch,dataset = "crowdhuman"):

        
    model = model.to(device)


    res = []
    i = 0
    for _ in range(1):
        for i, imgs in enumerate(test_loader):
            model.eval()
            imgs = list(img.to(device) for img in imgs)
            output = model(imgs)
            scores = output[0]['scores'].detach().cpu().numpy()
            num_people = len(scores[scores > 0.25])
            boxes = output[0]['boxes'].detach().cpu().numpy()
            boxes = boxes[:num_people]
            if len(boxes) > 0:
                boxes[:, [2, 3]] -= boxes[:, [0, 1]]

                for ii,box in enumerate(boxes):
                    temp = {}
                    temp['image_id'] = i+1
                    temp['category_id'] = 1
                    temp['bbox'] = box[:4].tolist()
                    temp['score'] = np.float(scores[ii])
                    res.append(temp)
                    

                print('\r%d/%d' % (i + 1, len(test_loader)), end='')
                sys.stdout.flush()
            

    with open('_temp_val\_temp_val_' + dataset + '_%d.json'%epoch, 'w') as f:
        json.dump(res, f)

# EWC

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import autograd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from barbar import Bar

In [14]:
class ElasticWeightConsolidation:

    def __init__(self, model, weight=1000000):
        self.model = model
        self.model.requires_grad_(True)
        self.weight = weight
        params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
        self.lr_scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer,
                                                step_size=3,
                                                gamma=0.1)


    def _update_mean_params(self):
        for param_name, param in self.model.named_parameters():
            _buff_param_name = param_name.replace('.', '__')
            self.model.register_buffer(_buff_param_name+'_estimated_mean', param.data.clone())

    def _update_fisher_params(self, current_ds, grad_avg_path = None):
        dl = current_ds
        model.requires_grad_(True)
        model.train()
        if grad_avg_path is not None:
            self.grad_avg = np.load(grad_avg_path, allow_pickle=True)
        else:
            i = 0
            for imgs, annotations in Bar(dl):
                imgs = list(img.to(device) for img in imgs)
                annotations = [{k:v.to(device) for k, v in t.items()} for t in annotations]

                train_flag = False
                if len(annotations[0]['boxes'].shape) == 2 or annotations[0]['boxes'].shape[-1] == 4:
                    if len(annotations) > 1:
                        if (len(annotations[1]['boxes'].shape) == 2 or annotations[1]['boxes'].shape[-1] == 4):
                            train_flag = True
                    else:
                        train_flag = True

                if train_flag:
                    loss_dict = self.model(imgs, annotations)
                    losses = sum(loss for loss in loss_dict.values())  
                    log_likelihood = torch.log(losses)
                    grad_log_liklihood = autograd.grad(log_likelihood, self.model.parameters())
                    grad_log_liklihood = [g.detach().cpu().numpy() for g in grad_log_liklihood]
                    # grad_list.append(grad_log_liklihood)

                    if i == 0:
                        grad_sum = [np.zeros(g.shape) for g in grad_log_liklihood]
                    
                    for ii in range(len(grad_log_liklihood)):
                        grad_sum[ii] += grad_log_liklihood[ii]

                    i += 1


            self.grad_avg = [g/i for g in grad_sum]
        
        _buff_param_names = [param[0].replace('.', '__') for param in self.model.named_parameters()]
        for _buff_param_name, param in zip(_buff_param_names, self.grad_avg):
            self.model.register_buffer(_buff_param_name+'_estimated_fisher', torch.tensor(param).data.clone() ** 2)

    def register_ewc_params(self, dataset, grad_avg_path = None):
        self._update_fisher_params(dataset, grad_avg_path)
        self._update_mean_params()

    def _compute_consolidation_loss(self, weight):
        try:
            losses = []
            for param_name, param in self.model.named_parameters():
                _buff_param_name = param_name.replace('.', '__')
                estimated_mean = getattr(self.model, '{}_estimated_mean'.format(_buff_param_name)).cuda()
                estimated_fisher = getattr(self.model, '{}_estimated_fisher'.format(_buff_param_name)).cuda()
                
                losses.append((estimated_fisher * (param - estimated_mean) ** 2).sum())
            return (weight / 2) * sum(losses)
        except AttributeError:
            return 0

    def forward_backward_update(self, input, target):
        output = self.model(input)
        loss = self._compute_consolidation_loss(self.weight) + self.crit(output, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def Fine_tune_new_data(self, Next_ds, epoch = 15):
        params = [p for p in self.model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.005,
                                    momentum=0.9, weight_decay=0.0005)

        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                        step_size=3,
                                                        gamma=0.1)

        epochs = 15
        best_epoch = 0
        self.model = self.model.to(device)
        min_loss = 10000
        for epoch in range(epochs):
            self.model.train()
            i = 0
            print("Epoch: %d/%d" % (epoch, epochs))
            loss_sum = 0
            for imgs, annotations in Bar(train_loader):
                i += 1
                imgs = list(img.to(device) for img in imgs)
                annotations = [{k:v.to(device) for k, v in t.items()} for t in annotations]
                
                train_flag = False
                if len(annotations[0]['boxes'].shape) == 2 or annotations[0]['boxes'].shape[-1] == 4:
                    if len(annotations) > 1:
                        if (len(annotations[1]['boxes'].shape) == 2 or annotations[1]['boxes'].shape[-1] == 4):
                            train_flag = True
                    else:
                        train_flag = True

                if train_flag:
                    loss_dict = model(imgs, annotations)
                    losses = self._compute_consolidation_loss(self.weight) + sum(loss for loss in loss_dict.values())
                    optimizer.zero_grad()
                    losses.backward()
                    optimizer.step()
                    loss_sum += losses.item()
                
            print("Iteration: {}; Loss: {}".format(i, loss_sum/i))
            if loss_sum/i < min_loss:
                min_loss = loss_sum/i
                best_epoch = epoch
                torch.save(model.state_dict(), 'ckpt/FRCNN_crowdhuman_EWC_CityPersons.pth')
                print("Model saved")
            print("Best epoch: {}; Best loss: {}".format(best_epoch, min_loss))
            lr_scheduler.step()
            log_output(model,test_loader_crowd,epoch,dataset = "crowdhuman")
            log_output(model,test_loader_city,epoch,dataset = "citypersons")

    def save(self, filename):
        torch.save(self.model, filename)

    def load(self, filename):
        self.model = torch.load(filename)

ewc = ElasticWeightConsolidation(model)

In [15]:
# ewc.register_ewc_params(train_loader,"Average_gradients\grad_avg_CrowdHuman.npy")
ewc.register_ewc_params(train_loader)

15000/15000: [===============================>] - ETA 4.3sssss


# Fine-tune with other dataset with EWC

In [16]:
class ImageDataset(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms=None):
    self.root = root
    self.transforms = transforms
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["file_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))
    # get annotation ID
    ann_ids = self.coco.getAnnIds(imgIds = img_id)
    # read annotation
    anns = self.coco.loadAnns(ann_ids)
    # num of people in the picture
    num_objs = len(anns)
    # build information about bounding box & area
    boxes = []
    areas = []
    for i in range(num_objs):
      x_min = anns[i]['bbox'][0]
      y_min = anns[i]['bbox'][1]
      x_max = x_min + anns[i]['bbox'][2]
      y_max = y_min + anns[i]['bbox'][3]
      boxes.append([x_min, y_min, x_max, y_max])
  
    # transfer information to Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.ones((num_objs,), dtype = torch.int64)
    img_id = torch.tensor([img_id])
    areas = torch.as_tensor(areas, dtype=torch.float32)
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    # Annotation in dict form
    Annotations = {
        "boxes" : boxes,
        "labels" : labels,
        "image_id" : img_id,
        "iscrowd" : iscrowd
    }

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img, Annotations
  
  def __len__(self):
    return len(self.ids)
    

def get_transforms(train):
  trans = []
  if train:
    trans.append(transforms.RandomHorizontalFlip(0.5))
    # trans.append(transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)))
    # trans.append(transforms.ColorJitter(brightness=1, contrast=1, saturation=1))
  trans.append(transforms.ToTensor())
  return transforms.Compose(trans)

train_data_path = r'data\citypersons\images'
coco_path = r"C:\Users\mahdi\Desktop\Pedestrian-Detection-master\data\citypersons\annotations\train.json"
train_dataset = ImageDataset(root=train_data_path, annotation=coco_path, transforms=get_transforms(train=True))
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))
    
train_loader = torch.utils.data.DataLoader(train_dataset, 2, shuffle=True, collate_fn=collate_fn)


loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [17]:
ewc.Fine_tune_new_data(train_loader)

Epoch: 0/15
2975/2975: [===============================>] - ETA 0.1sss
Iteration: 1488; Loss: 0.8074729018579866
Model saved
Best epoch: 0; Best loss: 0.8074729018579866
500/50070Epoch: 1/15
2975/2975: [===============================>] - ETA 0.4sss
Iteration: 1488; Loss: 0.7820223564011797
Model saved
Best epoch: 1; Best loss: 0.7820223564011797
499/50070Epoch: 2/15
2975/2975: [===============================>] - ETA 0.3sss
Iteration: 1488; Loss: 0.7661525166340333
Model saved
Best epoch: 2; Best loss: 0.7661525166340333
500/50070Epoch: 3/15
2975/2975: [===============================>] - ETA 0.3sss
Iteration: 1488; Loss: 0.7062257246322047
Model saved
Best epoch: 3; Best loss: 0.7062257246322047
500/50070Epoch: 4/15
2975/2975: [===============================>] - ETA 0.3sss
Iteration: 1488; Loss: 0.6896272712810747
Model saved
Best epoch: 4; Best loss: 0.6896272712810747
499/50070Epoch: 5/15
2975/2975: [===============================>] - ETA 0.3sss
Iteration: 1488; Loss: 0.688249578

In [18]:
#Load the weights of the model
model.load_state_dict(torch.load('ckpt\FRCNN_crowdhuman_EWC_CityPersons.pth'))
model = model.to(device)

In [19]:
from utils.eval_demo import validate

def val(gt_json_path, pred_json_path, log=None):
    MRs = validate(gt_json_path, pred_json_path)
    print('Summarize: [Reasonable: %.2f%%], [Bare: %.2f%%], [Partial: %.2f%%], [Heavy: %.2f%%]'
          % (MRs[0]*100, MRs[1]*100, MRs[2]*100, MRs[3]*100))
    if log is not None:
        log.write("%.7f %.7f %.7f %.7f\n" % tuple(MRs))
    return MRs[0]

In [20]:
for epoch in range(15):
    print(epoch)
    val(r"C:\Users\mahdi\Desktop\Pedestrian-Detection-master\val_crowdhuman.json", r'_temp_val\_temp_val_crowdhuman_%d.json'%epoch)
    val(r"data\citypersons\annotations\val_gt.json", r'_temp_val\_temp_val_citypersons_%d.json'%epoch)

0
Summarize: [Reasonable: 61.67%], [Bare: 47.85%], [Partial: 64.99%], [Heavy: 87.35%]
Summarize: [Reasonable: 36.72%], [Bare: 27.95%], [Partial: 41.69%], [Heavy: 80.12%]
1
Summarize: [Reasonable: 63.53%], [Bare: 50.28%], [Partial: 62.65%], [Heavy: 81.72%]
Summarize: [Reasonable: 32.52%], [Bare: 22.98%], [Partial: 37.24%], [Heavy: 75.33%]
2
Summarize: [Reasonable: 69.79%], [Bare: 59.33%], [Partial: 69.34%], [Heavy: 86.26%]
Summarize: [Reasonable: 47.21%], [Bare: 38.12%], [Partial: 52.35%], [Heavy: 79.52%]
3
Summarize: [Reasonable: 59.88%], [Bare: 46.45%], [Partial: 58.66%], [Heavy: 81.56%]
Summarize: [Reasonable: 24.58%], [Bare: 17.52%], [Partial: 26.99%], [Heavy: 66.35%]
4
Summarize: [Reasonable: 58.41%], [Bare: 44.82%], [Partial: 57.24%], [Heavy: 80.45%]
Summarize: [Reasonable: 24.89%], [Bare: 17.74%], [Partial: 27.39%], [Heavy: 67.22%]
5
Summarize: [Reasonable: 58.51%], [Bare: 45.09%], [Partial: 57.63%], [Heavy: 81.00%]
Summarize: [Reasonable: 24.65%], [Bare: 17.22%], [Partial: 27.05

In [16]:
#test data loader fo citypersons
class ImageDataset_test(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms):
    self.root = root
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))
    self.transforms = transforms

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["im_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img
  
  def __len__(self):
    return len(self.ids)

test_data_path = r'data\citypersons\images\val'
coco_path_test = r"data\citypersons\annotations\val_gt.json"
test_dataset = ImageDataset_test(root=test_data_path, annotation=coco_path_test, transforms=get_transforms(train=False))

    
test_loader = torch.utils.data.DataLoader(test_dataset, 1, shuffle=False)

import sys
import json
from IPython.display import clear_output
model = model.to(device)

res = []
i = 0
for _ in range(1):
    for i, imgs in enumerate(test_loader):
        model.eval()
        imgs = list(img.to(device) for img in imgs)
        output = model(imgs)
        scores = output[0]['scores'].detach().cpu().numpy()
        num_people = len(scores[scores > 0.25])
        boxes = output[0]['boxes'].detach().cpu().numpy()
        boxes = boxes[:num_people]
        if len(boxes) > 0:
            boxes[:, [2, 3]] -= boxes[:, [0, 1]]

            for ii,box in enumerate(boxes):
                temp = {}
                temp['image_id'] = i+1
                temp['category_id'] = 1
                temp['bbox'] = box[:4].tolist()
                temp['score'] = np.float(scores[ii])
                res.append(temp)
                

            print('\r%d/%d' % (i + 1, len(test_loader)), end='')
            sys.stdout.flush()
        

with open('_temp_val/_temp_val_crowdhuman_EWC_citypersons.json', 'w') as f:
    json.dump(res, f)


res = val(r"data\citypersons\annotations\val_gt.json", r'_temp_val/_temp_val_crowdhuman_EWC_citypersons.json')
print(res)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
500/500

FileNotFoundError: [Errno 2] No such file or directory: '_temp_eval/_temp_val_crowdhuman_EWC_citypersons.json'

In [19]:
#test data loader
class ImageDataset_test(torch.utils.data.Dataset):
  def __init__(self, root, annotation, transforms):
    self.root = root
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))
    self.transforms = transforms

  def __getitem__(self, idx):
    # image ID
    img_id = self.ids[idx]
    # image file_name
    img_file = self.coco.loadImgs(img_id)[0]["file_name"]
    # read_imgae
    img = Image.open(os.path.join(self.root, img_file))

    # transforms
    if self.transforms is not None:
      img = self.transforms(img)
    
    return img
  
  def __len__(self):
    return len(self.ids)

test_data_path = r'data\CrowdHuman\images\val'
coco_path_test = r"data\CrowdHuman\val.json"
test_dataset = ImageDataset_test(root=test_data_path, annotation=coco_path_test, transforms=get_transforms(train=False))

    
test_loader = torch.utils.data.DataLoader(test_dataset, 1, shuffle=False)

import sys
import json
from IPython.display import clear_output
model = model.to(device)

res = []
i = 0
for _ in range(1):
    for i, imgs in enumerate(test_loader):
        model.eval()
        imgs = list(img.to(device) for img in imgs)
        output = model(imgs)
        scores = output[0]['scores'].detach().cpu().numpy()
        num_people = len(scores[scores > 0.25])
        boxes = output[0]['boxes'].detach().cpu().numpy()
        boxes = boxes[:num_people]
        if len(boxes) > 0:
            boxes[:, [2, 3]] -= boxes[:, [0, 1]]

            for ii,box in enumerate(boxes):
                temp = {}
                temp['image_id'] = i+1
                temp['category_id'] = 1
                temp['bbox'] = box[:4].tolist()
                temp['score'] = np.float(scores[ii])
                res.append(temp)
                

            print('\r%d/%d' % (i + 1, len(test_loader)), end='')
            sys.stdout.flush()
        

with open('_temp_val/_temp_val_crowdhuman_EWC_citypersons.json', 'w') as f:
    json.dump(res, f)


res = val(r"C:\Users\mahdi\Desktop\Pedestrian-Detection-master\val_crowdhuman.json", r'_temp_val/_temp_val_crowdhuman_EWC_citypersons.json')
print(res)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!
4370/4370Summarize: [Reasonable: 58.51%], [Bare: 45.56%], [Partial: 57.54%], [Heavy: 81.00%]
0.5850768791588195


In [ ]:
from eval_city.eval_script.eval_demo import validate

def val(gt_json_path, pred_json_path, log=None):
    MRs = validate(gt_json_path, pred_json_path)
    print('Summarize: [Reasonable: %.2f%%], [Bare: %.2f%%], [Partial: %.2f%%], [Heavy: %.2f%%]'
          % (MRs[0]*100, MRs[1]*100, MRs[2]*100, MRs[3]*100))
    if log is not None:
        log.write("%.7f %.7f %.7f %.7f\n" % tuple(MRs))
    return MRs[0]